<a href="https://colab.research.google.com/github/lem6ns/colab-swiss-army-knife/blob/master/lemons'_swiss_army_knife_for_Colab.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# lemons' Swiss Army Knife for Google Colab

## Mounting Google Drive
This is important, you should run this every time you use this notebook.

In [ ]:
#@markdown <br><center><img src='https://upload.wikimedia.org/wikipedia/commons/thumb/d/da/Google_Drive_logo.png/600px-Google_Drive_logo.png' height="50" alt="Gdrive-logo"/></center>
#@markdown <center><h3>Mount GDrive to /content/drive</h3></center><br>
MODE = "MOUNT" #@param ["MOUNT", "UNMOUNT"]
#Mount your Gdrive! 
from google.colab import drive
drive.mount._DEBUG = False
if MODE == "MOUNT":
  drive.mount('/content/drive', force_remount=True)
elif MODE == "UNMOUNT":
  try:
    drive.flush_and_unmount()
  except ValueError:
    pass
  get_ipython().system_raw("rm -rf /root/.config/Google/DriveFS")

## Torrent to Google Drive
credits to [Rishabh Singh](https://github.com/r12habh)

### Installing dependencies
Run this to install the required dependencies.

In [ ]:
!python -m pip install --upgrade pip setuptools wheel && python -m pip install lbry-libtorrent && apt install python3-libtorrent

     |████████████████████████████████| 2.1 MB 5.5 MB/s 
     |████████████████████████████████| 1.1 MB 43.2 MB/s 
  Attempting uninstall: setuptools
    Found existing installation: setuptools 57.4.0
    Uninstalling setuptools-57.4.0:
      Successfully uninstalled setuptools-57.4.0
  Attempting uninstall: pip
    Found existing installation: pip 21.1.3
    Uninstalling pip-21.1.3:
      Successfully uninstalled pip-21.1.3
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tensorflow 2.8.0 requires tf-estimator-nightly==2.8.0.dev2021122109, which is not installed.
datascience 0.10.6 requires folium==0.2.1, but you have folium 0.8.3 which is incompatible.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 30.0 MB/s eta 0:00:00
Reading package lists... Done
Building dependency tree       
Reading state information... Done
The following package was automatica

### Torrenting
Change the settings to your needs, and run it.

In [ ]:
#@markdown <br><center><img src='https://www.goodfreephotos.com/albums/vector-images/download-icon-vector-clipart.png' height="50" alt="Download Icon"/></center>
#@markdown <center><h3>Torrent to Google Drive</h3></center><br>
import libtorrent as lt
import time
import datetime
outputDirectory = "/content/drive/" #@param {type:"string"} 

def download_torrent():
    ses = lt.session()
    ses.listen_on(6881, 6891)
    #@markdown <center><p>Input a Magnet Link or a Direct Link to a .torrent.</[]></center><br>
    link = "" #@param {type:"string"}
    print(link)

    handle = lt.add_magnet_uri(ses, link, params)
    # change the 0 to a 1 to download sequential - this sequential option is only if you selected zip. If not,
    # scroll farther down.
    #@markdown <center><p>Change this to "1" if you are zipping the torrent up.</p></center><br>
    sequentialDownload = 0 #@param {type:"slider", min:0, max:1, step:1}
    handle.set_sequential_download(0) 

    ses.start_dht()
    begin = time.time()
    print(datetime.datetime.now())

    print('Downloading Metadata...')
    while not handle.has_metadata():
        time.sleep(1)
    print('Got Metadata, Starting Torrent Download...')

    print("Starting", handle.name())

    while handle.status().state != lt.torrent_status.seeding:
        s = handle.status()
        state_str = ['queued', 'checking', 'downloading metadata',
                     'downloading', 'finished', 'seeding', 'allocating', 'checking fastresume']
        print('%.2f%% complete (down: %.1f kb/s up: %.1f kB/s peers: %d) %s ' % \
              (s.progress * 100, s.download_rate / 1000, s.upload_rate / 1000, \
               s.num_peers, state_str[s.state]))
        time.sleep(5)

    end = time.time()
    print(handle.name(), "COMPLETE")

    print("Elapsed Time: ", int((end - begin) // 60), "min :", int((end - begin) % 60), "sec")

    print(datetime.datetime.now())


zipp = "No" #@param ["Yes", "No"]

if zipp == 'Yes':
    zip_name = "" #@param {type:"string"}
    import shutil

    params = {
        'save_path': '/content/temp/',
        'storage_mode': lt.storage_mode_t(2),
    }
    download_torrent()

    print("now time to zip")

    print("Zipping ...")
    final_zip_directory = outputDirectory + "/" + zip_name
    shutil.make_archive(final_zip_directory, 'zip', '/content/temp/')
    shutil.rmtree('/content/temp/')

else:
    params = {
        'save_path': outputDirectory,
        'storage_mode': lt.storage_mode_t(2),
    }
    download_torrent()

print('\nALL DONE!')


## Transfer from Mega to Google Drive
credits to [cheems](https://github.com/cheems)

### Have a upgraded MEGA Account?

In [ ]:
# @markdown <center><h3>Input Your Mega ID</h3></center><br>
from functools import wraps
import errno
import os
import signal
import subprocess
import shlex


class TimeoutError(Exception):
    pass


def timeout(seconds=10, error_message=os.strerror(errno.ETIME)):
    def decorator(func):
        def _handle_timeout(signum, frame):
            raise TimeoutError(error_message)

        def wrapper(*args, **kwargs):
            signal.signal(signal.SIGALRM, _handle_timeout)
            signal.alarm(seconds)
            try:
                result = func(*args, **kwargs)
            finally:
                signal.alarm(0)
            return result

        return wraps(func)(wrapper)

    return decorator


if not os.path.exists("/root/.ipython/ocr.py"):
    from subprocess import run
    from shlex import split

    shellCmd = "wget -qq https://raw.githubusercontent.com/biplobsd/OneClickRun/master/res/ocr.py \
                    -O /root/.ipython/ocr.py"
    run(split(shellCmd))
from ocr import runSh

@timeout(10)
def runShT(args):
    return runSh(args, output=True)


# MEGAcmd installing
if not os.path.exists("/usr/bin/mega-cmd"):
    print("Installing MEGA ...")
    runSh('sudo apt-get -y update')
    runSh('sudo apt-get -y install libmms0 libc-ares2 libc6 libcrypto++6 libgcc1 libmediainfo0v5 libpcre3 libpcrecpp0v5 libssl1.1 libstdc++6 libzen0v5 zlib1g apt-transport-https')
    runSh('sudo curl -sL -o /var/cache/apt/archives/MEGAcmd.deb https://mega.nz/linux/MEGAsync/Debian_9.0/amd64/megacmd-Debian_9.0_amd64.deb', output=True)
    runSh('sudo dpkg -i /var/cache/apt/archives/MEGAcmd.deb', output=True)
    print("MEGA is installed.")
else:
    !pkill mega-cmd

# INPUT YOUR MEGA ID

USERNAME = ""  # @param {type:"string"}
PASSWORD = ""  # @param {type:"string"}
if not (USERNAME == "" or PASSWORD == ""):
    try:
        runShT(f"mega-login {USERNAME} {PASSWORD}")
    except TimeoutError:
        runSh('mega-whoami', output=True)
else:
    print("Please Input your Mega IDs.")
# @markdown *After signing in, use the cell below to use the quota of the account above.*

### Public Link Transfer

In [ ]:
import sys, os, urllib.request
import time
import subprocess
import contextlib
from IPython.display import clear_output
#@markdown <br><center><img src='https://mega.nz/favicon.ico?v=3' height="50" alt="MEGA-logo"/></center>
#@markdown <center><h2>Transfer from Mega to Google Drive</h2></center><br>
HOME = os.path.expanduser("~")
if not os.path.exists(f"{HOME}/.ipython/ocr.py"):
    hCode = "https://raw.githubusercontent.com/biplobsd/" \
                "OneClickRun/master/res/ocr.py"
    urllib.request.urlretrieve(hCode, f"{HOME}/.ipython/ocr.py")

from ocr import (
    runSh,
    loadingAn,
)
URL = "" #@param {type:"string"}
OUTPUT_PATH = "" #@param {type:"string"}
#@markdown #####_*Sometimes this cell doesn't stop itself after the completion of the transfer. In case of that stop the cell manually._
if not OUTPUT_PATH:
  os.makedirs("downloads", exist_ok=True)
  OUTPUT_PATH = "downloads"
# MEGAcmd installing
if not os.path.exists("/usr/bin/mega-cmd"):
    loadingAn()
    print("Installing MEGA ...")
    runSh('sudo apt-get -y update')
    runSh('sudo apt-get -y install libmms0 libc-ares2 libc6 libcrypto++6 libgcc1 libmediainfo0v5 libpcre3 libpcrecpp0v5 libssl1.1 libstdc++6 libzen0v5 zlib1g apt-transport-https')
    runSh('sudo curl -sL -o /var/cache/apt/archives/MEGAcmd.deb https://mega.nz/linux/MEGAsync/Debian_9.0/amd64/megacmd-Debian_9.0_amd64.deb', output=True)
    runSh('sudo dpkg -i /var/cache/apt/archives/MEGAcmd.deb', output=True)
    print("MEGA is installed.")
    clear_output()

# Unix, Windows and old Macintosh end-of-line
newlines = ['\n', '\r\n', '\r']

def unbuffered(proc, stream='stdout'):
    stream = getattr(proc, stream)
    with contextlib.closing(stream):
        while True:
            out = []
            last = stream.read(1)
            # Don't loop forever
            if last == '' and proc.poll() is not None:
                break
            while last not in newlines:
                # Don't loop forever
                if last == '' and proc.poll() is not None:
                    break
                out.append(last)
                last = stream.read(1)
            out = ''.join(out)
            yield out


def transfare():
    import codecs
    decoder = codecs.getincrementaldecoder("UTF-8")()
    cmd = ["mega-get", URL, OUTPUT_PATH]
    proc = subprocess.Popen(
        cmd,
        stdout=subprocess.PIPE,
        stderr=subprocess.STDOUT,
        # Make all end-of-lines '\n'
        universal_newlines=True,
    )
    for line in unbuffered(proc):
        print(line)
        


transfare()



## Transfer from Google Drive to Mega
credits to [Suraj Bahadur](https://github.com/surajbahadur591)

### Sign In

In [ ]:
# @markdown <center><h2>Input your Mega ID and Password</h2></center><br>

# https://github.com/surajbahadur591

from functools import wraps
import errno
import os
import signal
import subprocess
import shlex


class TimeoutError(Exception):
    pass


def timeout(seconds=10, error_message=os.strerror(errno.ETIME)):
    def decorator(func):
        def _handle_timeout(signum, frame):
            raise TimeoutError(error_message)

        def wrapper(*args, **kwargs):
            signal.signal(signal.SIGALRM, _handle_timeout)
            signal.alarm(seconds)
            try:
                result = func(*args, **kwargs)
            finally:
                signal.alarm(0)
            return result

        return wraps(func)(wrapper)

    return decorator


if not os.path.exists("/root/.ipython/ocr.py"):
    from subprocess import run
    from shlex import split

    shellCmd = "wget -qq https://raw.githubusercontent.com/biplobsd/OneClickRun/master/res/ocr.py \
                    -O /root/.ipython/ocr.py"
    run(split(shellCmd))
from ocr import runSh

@timeout(10)
def runShT(args):
    return runSh(args, output=True)


# MEGAcmd installing
if not os.path.exists("/usr/bin/mega-cmd"):
    print("Installing MEGA ...")
    runSh('sudo apt-get -y update')
    runSh('sudo apt-get -y install libmms0 libc-ares2 libc6 libcrypto++6 libgcc1 libmediainfo0v5 libpcre3 libpcrecpp0v5 libssl1.1 libstdc++6 libzen0v5 zlib1g apt-transport-https')
    runSh('sudo curl -sL -o /var/cache/apt/archives/MEGAcmd.deb https://mega.nz/linux/MEGAsync/Debian_9.0/amd64/megacmd-Debian_9.0_amd64.deb', output=True)
    runSh('sudo dpkg -i /var/cache/apt/archives/MEGAcmd.deb', output=True)
    print("MEGA is installed.")
else:
    !pkill mega-cmd

# INPUT YOUR MEGA ID

USERNAME = ""  # @param {type:"string"}
PASSWORD = ""  # @param {type:"string"}
if not (USERNAME == "" or PASSWORD == ""):
    try:
        runShT(f"mega-login {USERNAME} {PASSWORD}")
    except TimeoutError:
        runSh('mega-whoami', output=True)
else:
    print("Please Input your Mega ID.")


### Transfer

In [ ]:
import time
import subprocess
import contextlib
from IPython.display import clear_output
# @markdown <br><center><img src='https://i.imgur.com/UjCRbEF.png' height="300" alt="netdata"/></center>
# @markdown <center><h2>TRANSFER NOW</h2></center><br>
# TRANSFER NOW

PATH_TO_FILE = "/content/drive/"  # @param {type:"string"}
NAME_OF_FILE_ON_MEGA = ""  # @param {type:"string"}
# Unix, Windows and old Macintosh end-of-line
newlines = ['\n', '\r\n', '\r']


def unbuffered(proc, stream='stdout'):
    stream = getattr(proc, stream)
    with contextlib.closing(stream):
        while True:
            out = []
            last = stream.read(1)
            # Don't loop forever
            if last == '' and proc.poll() is not None:
                break
            while last not in newlines:
                # Don't loop forever
                if last == '' and proc.poll() is not None:
                    break
                out.append(last)
                last = stream.read(1)
            out = ''.join(out)
            yield out


def transfare():
    cmd = ['mega-put', PATH_TO_FILE, "/" + NAME_OF_FILE_ON_MEGA]
    proc = subprocess.Popen(
        cmd,
        stdout=subprocess.PIPE,
        stderr=subprocess.STDOUT,
        # Make all end-of-lines '\n'
        universal_newlines=True,
    )
    for line in unbuffered(proc):
        clear_output(wait=True)
        print(line)

try:
    transfare()
except FileNotFoundError:
    print("Login your account!")


## Compression
credits to [cheems](https://github.com/cheems)

### Compression

#### ZIP File

In [ ]:
#@markdown <br><center><img src='https://icons.iconarchive.com/icons/hopstarter/soft-scraps/256/Folder-Compressed-icon.png' height="50" alt="Gdrive-logo"/></center>
#@markdown <center><h3>Compress ZIP File</h3></center><br>

OUTPUT_FILE_PATH = "/content/drive/file.zip" #@param {type:"string"}
INPUT_FOLDER_OR_FILE_PATH = "/content/drive/" #@param {type:"string"}
!zip OUTPUT_FILE_PATH INPUT_FOLDER_OR_FILE_PATH

#### RAR File

##### Dependency

In [ ]:
#@markdown <br><center><img src='https://icons.iconarchive.com/icons/hopstarter/soft-scraps/256/Folder-Compressed-icon.png' height="50" alt="Gdrive-logo"/></center>
#@markdown <center><h3>Dependency</h3></center><br>
#@markdown <center><p>Run this cell!</p></center><br>
!apt-get install rar

##### Not Password Protected

In [ ]:
#@markdown <br><center><img src='https://icons.iconarchive.com/icons/hopstarter/soft-scraps/256/Folder-Compressed-icon.png' height="50" alt="Gdrive-logo"/></center>
#@markdown <center><h3>Compress RAR File</h3></center><br>

OUTPUT_FILE_PATH = "/content/drive/file.rar" #@param {type:"string"}
INPUT_FOLDER_OR_FILE_PATH = "/content/drive/" #@param {type:"string"}
!rar a OUTPUT_FILE_PATH INPUT_FOLDER_OR_FILE_PATH

##### Password Protected

In [ ]:
#@markdown <br><center><img src='https://icons.iconarchive.com/icons/hopstarter/soft-scraps/256/Folder-Compressed-icon.png' height="50" alt="Gdrive-logo"/></center>
#@markdown <center><h3>Compress RAR File</h3></center><br>

OUTPUT_FILE_PATH = "/content/drive/file.rar" #@param {type:"string"}
INPUT_FOLDER_OR_FILE_PATH = "/content/drive/" #@param {type:"string"}
!rar a -p OUTPUT_FILE_PATH INPUT_FOLDER_OR_FILE_PATH

#### TAR File

##### .tar

In [ ]:
#@markdown <br><center><img src='https://icons.iconarchive.com/icons/hopstarter/soft-scraps/256/Folder-Compressed-icon.png' height="50" alt="Gdrive-logo"/></center>
#@markdown <center><h3>Compress TAR File</h3></center><br>

OUTPUT_FILE_PATH = "/content/drive/file.tar" #@param {type:"string"}
INPUT_FOLDER_OR_FILE_PATH = "/content/drive/" #@param {type:"string"}
!tar -cvf OUTPUT_FILE_PATH INPUT_FOLDER_OR_FILE_PATH

##### .tar.gz

In [ ]:
#@markdown <br><center><img src='https://icons.iconarchive.com/icons/hopstarter/soft-scraps/256/Folder-Compressed-icon.png' height="50" alt="Gdrive-logo"/></center>
#@markdown <center><h3>Compress TAR.GZ File</h3></center><br>

OUTPUT_FILE_PATH = "/content/drive/file.tar.gz" #@param {type:"string"}
INPUT_FOLDER_OR_FILE_PATH = "/content/drive/" #@param {type:"string"}
!tar -czvf OUTPUT_FILE_PATH INPUT_FOLDER_OR_FILE_PATH

##### tar.bz2

In [ ]:
#@markdown <br><center><img src='https://icons.iconarchive.com/icons/hopstarter/soft-scraps/256/Folder-Compressed-icon.png' height="50" alt="Gdrive-logo"/></center>
#@markdown <center><h3>Compress TAR.BZ2 File</h3></center><br>

OUTPUT_FILE_PATH = "/content/drive/file.tar.bz2" #@param {type:"string"}
INPUT_FOLDER_OR_FILE_PATH = "/content/drive/" #@param {type:"string"}
!tar -cjvf OUTPUT_FILE_PATH INPUT_FOLDER_OR_FILE_PATH

### Decompression

#### ZIP File

In [ ]:
#@markdown <br><center><img src='https://icons.iconarchive.com/icons/visualpharm/must-have/256/Open-icon.png' height="50" alt="Gdrive-logo"/></center>
#@markdown <center><h3>Decompress ZIP File</h3></center><br>

INPUT_FILE_PATH = "/content/drive/file.zip" #@param {type:"string"}
OUTPUT_PATH = "/content/drive/" #@param {type:"string"}
!unzip INPUT_FILE_PATH -d OUTPUT_PATH

#### RAR File

In [ ]:
#@markdown <br><center><img src='https://icons.iconarchive.com/icons/visualpharm/must-have/256/Open-icon.png' height="50" alt="Gdrive-logo"/></center>
#@markdown <center><h3>Decompress RAR File</h3></center><br>

INPUT_FILE_PATH = "/content/drive/file.rar" #@param {type:"string"}
OUTPUT_PATH = "/content/drive/" #@param {type:"string"}
!unrar x INPUT_FILE_PATH OUTPUT_PATH

##### Password Protected

In [ ]:
#@markdown <br><center><img src='https://icons.iconarchive.com/icons/visualpharm/must-have/256/Open-icon.png' height="50" alt="Gdrive-logo"/></center>
#@markdown <center><h3>Decompress RAR File</h3></center><br>

INPUT_FILE_PATH = "/content/drive/" #@param {type:"string"}
OUTPUT_PATH = "/content/drive/" #@param {type:"string"}
!unrar x -Y INPUT_FILE_PATH OUTPUT_PATH

#### TAR File

##### .tar

In [ ]:
#@markdown <br><center><img src='https://icons.iconarchive.com/icons/visualpharm/must-have/256/Open-icon.png' height="50" alt="Gdrive-logo"/></center>
#@markdown <center><h3>Decompress TAR File</h3></center><br>

INPUT_FILE_PATH = "/content/drive/file.tar" #@param {type:"string"}
OUTPUT_PATH = "/content/drive/" #@param {type:"string"}
!tar -xvf INPUT_FILE_PATH -C OUTPUT_PATH

##### .tar.gz

In [ ]:
#@markdown <br><center><img src='https://icons.iconarchive.com/icons/visualpharm/must-have/256/Open-icon.png' height="50" alt="Gdrive-logo"/></center>
#@markdown <center><h3>Decompress TAR.GZ File</h3></center><br>

INPUT_FILE_PATH = "/content/drive/file.tar.gz" #@param {type:"string"}
OUTPUT_PATH = "/content/drive/" #@param {type:"string"}
!tar -xzvf INPUT_FILE_PATH -C OUTPUT_PATH

##### .tar.bz2

In [ ]:
#@markdown <br><center><img src='https://icons.iconarchive.com/icons/visualpharm/must-have/256/Open-icon.png' height="50" alt="Gdrive-logo"/></center>
#@markdown <center><h3>Decompress TAR.BZ2 File</h3></center><br>

INPUT_FILE_PATH = "/content/drive/file.tar.bz2" #@param {type:"string"}
OUTPUT_PATH = "/content/drive/" #@param {type:"string"}
!tar -xjvf INPUT_FILE_PATH -C OUTPUT_PATH